
## ENSAE - 2A - Econometrics 1 - TD9 


In [2]:
install.packages("dplyr")
library(haven)
library(dplyr)

# Set the working directory
#setwd("~//Documents/Econo1/TD9/TD9")
data <- read_dta("jtpa.dta")

Mise à jour de la liste HTML des packages dans '.Library'

Making 'packages.html' ...
 terminé



1. Estimate the average eﬀect of the treatment on the compliers : δC = E(Y (1) − Y (0)|C).
Does the training given in the JTPA program seem eﬀective ?

In [3]:
  
means <- data %>%group_by(z) %>%summarise(mu = mean(y, na.rm = TRUE),d_mean = mean(d, na.rm = TRUE))

# Extract the means
mu1 <- means$mu[means$z == 1]
mu0 <- means$mu[means$z == 0]
d1 <- means$d_mean[means$z == 1]
d0 <- means$d_mean[means$z == 0]

# Compute deltac
deltac <- (mu1 - mu0) / (d1 - d0)
deltac

[1] 1848.829

2. Compute the probabilities P (C), P (AT ), and P (N T ) as a function of the distribution of
observed variables. Is it possible, in some situations, to reject the hypotheses of monoto-
nicity and independence ? Estimate the probabilities for the three groups in the data.

In [4]:
PAT <- d0
PNT <- 1-d1
PC <- 1-PAT-PNT

print(PAT)
print(PNT)
print(PC)
#PC is positive, so we do not directly reject the monotonicity assumption

[1] 0.01452785
[1] 0.3583545
[1] 0.6271177


3. To better know who are the compliers, for some discrete random variable X, we study
the quantities g(x) = P (X = x | C)/P (X = x).
Under the assumption that Z ⊥⊥ (X, Y (0), Y (1), D(0), D(1)), write g(x) as a function of
the distributions of the observed variables (D, Z, X).
Estimate g(x) in the following cases : X is the gender, X is the ethnic group, X is the
level of education 2, and X is age categories. Comment on the results and describe the
population of compliers

In [5]:
########################### Gender ######################################
means_sex <- data %>%
group_by(z, sex) %>%
summarise(mean_d = mean(d, na.rm = TRUE), .groups = "drop")

# Extract relevant means
woman_0 <- means_sex %>% filter(z == 0, sex == 0) %>% pull(mean_d)
man_0 <- means_sex %>% filter(z == 0, sex == 1) %>% pull(mean_d)
woman_1 <- means_sex %>% filter(z == 1, sex == 0) %>% pull(mean_d)
man_1 <- means_sex %>% filter(z == 1, sex == 1) %>% pull(mean_d)

# Calculate g_woman and g_man
g_woman <- (woman_1 - woman_0) / PC
g_man <- (man_1 - man_0) / PC

# Display results
print(g_woman)
print(g_man)

########################### Operational function #################################

#Function to automatize the computation of g(x)
calculate_g <- function(data, group_var, PC) {
  # Dynamically group by z and the specified variable
  means <- data %>%
    group_by(z, !!sym(group_var)) %>%  # Dynamically pass column name
    summarise(mean_d = mean(d, na.rm = TRUE), .groups = "drop")
  
  # Extract means for the group_var == 1 case
  var_0 <- means %>% filter(z == 0, !!sym(group_var) == 1) %>% pull(mean_d)
  var_1 <- means %>% filter(z == 1, !!sym(group_var) == 1) %>% pull(mean_d)
  #This function does not work for the case of women, would need to change group_var to 0
  
  # Compute the scaled difference
  g_var <- (var_1 - var_0) / PC
  return(g_var)
}

########################### Ethnicity ######################################

calculate_g(data,'black', PC)
calculate_g(data,'hispanic', PC)

########################### Level of education ######################################

calculate_g(data,'hsorged', PC)

########################### Age group ######################################

calculate_g(data,'age2225', PC)
calculate_g(data,'age2629', PC)
calculate_g(data,'age3035', PC)
calculate_g(data,'age3644', PC)
calculate_g(data,'age4554', PC)

[1] 1.020401
[1] 0.9753727


[1] 0.9470811

[1] 1.095722

[1] 1.028798

[1] 1.031896

[1] 0.9854354

[1] 1.03401

[1] 0.9688944

[1] 0.903947

4. We deﬁne δAT := E(Y (1) − Y (0)|AT ), δT := E(Y (1) − Y (0)|D = 1),
and λ := P (C, Z = 1)/[P (C, Z = 1) + P (AT )]. Show the following equality :
δT = λδC + (1 − λ)δAT
Estimate λ in the data. Explain why it is sensible here to assume that (δT − δC )/δC is
close to 0.

In [6]:
  
# Summary statistics for d
mean_d <- mean(data$d, na.rm = TRUE)

# Calculate lambda
lambda <- (mean_d - PAT) / mean_d

print(lambda)


[1] 0.9664944


5. 

In [8]:
  
reg_model <- lm(y ~ d, data = data)
betad <- reg_model$coefficients['d']

B <- betad - deltac
B

d 
942.2585